## Modified RNN or LSTM model for Sentiment Analysis of  Amazon reviews

In [2]:
import numpy as np
import pandas as pd
import os
import re
import nltk
from nltk.corpus import stopwords
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers.core import Activation, Dropout, Dense
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer

Using TensorFlow backend.


In [4]:
os.chdir("C:\\Users\Venkateswara Rao\\Desktop")

In [5]:
Amazon=pd.read_csv("amazon_cells_labelled.txt", delimiter="\t", header=None, names=["review","label"])


In [6]:
Amazon.head()

,review,label
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1


In [7]:
### Data Frame Attributes 
Amazon.ndim

2

In [8]:
Amazon.shape

(1000, 2)

In [9]:
Amazon["label"].value_counts() 

1    500
0    500
Name: label, dtype: int64

In [10]:
import nltk 
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
lematizer=WordNetLemmatizer() 

## Preprocessing 

In [11]:
X=[]
for i in range(0,len(Amazon)):
    review=re.sub('[^a-zA-Z]', ' ', Amazon['review'][i])## # Remove  Numbers and special characters( punctuations) 
    review=re.sub(r'\s+[a-zA-Z]\s+', ' ', Amazon['review'][i])### Single character removal
    review=re.sub(r'\s+', ' ', Amazon['review'][i])### Removing multiple spaces
    review=review.lower()
    review=review.split()
    review=[lematizer.lemmatize(word) for word in review if not word in stopwords.words('english')]
    review=' '.join(review)
    X.append(review)

In [12]:
X[0:5]

['way plug u unless go converter.',
 'good case, excellent value.',
 'great jawbone.',
 'tied charger conversation lasting 45 minutes.major problems!!',
 'mic great.']

In [13]:
y=Amazon['label']

In [14]:
### Spliting Train and test datasets
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.20,random_state=123)

## Preparing the Embedding Layer

In [15]:
tokenizer = Tokenizer(num_words=1500)
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [16]:
## truncate and pad input sequences.
# Adding 1 because of reserved 0 index
vocab_size = len(tokenizer.word_index) + 1

maxlen = 100

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

## Create Modified RNN or  LSTM model

In [17]:
# create the model
embedding_vector_length = 32
model = Sequential()

In [18]:
model.add(Embedding(vocab_size, embedding_vector_length,\
                       input_length=maxlen)) # Embedding is euivalent to one hot encoding with lesser dimentionality



In [19]:
model.add(LSTM(100))

In [20]:
model.add(Dense(1, activation='sigmoid'))


In [21]:
model.compile(loss='binary_crossentropy',
                 optimizer='adam',
                   metrics=['accuracy'])

In [22]:
model.fit(X_train, y_train,
     validation_data=(X_test, y_test),
           epochs=3, batch_size=64)


C:\Users\Venkateswara Rao\Anaconda3\envs\venkat\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 800 samples, validate on 200 samples
Epoch 1/3
800/800 [==============================] - 4s 4ms/step - loss: 0.6945 - accuracy: 0.5013 - val_loss: 0.6933 - val_accuracy: 0.4950
Epoch 2/3
800/800 [==============================] - 3s 4ms/step - loss: 0.6932 - accuracy: 0.5013 - val_loss: 0.6931 - val_accuracy: 0.5050
Epoch 3/3
800/800 [==============================] - 3s 4ms/step - loss: 0.6934 - accuracy: 0.4988 - val_loss: 0.6931 - val_accuracy: 0.5050


In [23]:
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)


In [24]:
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 50.50%
